# L08.1 Similarity Practice
---
이름: 김태산

학과: 자동차공학과

학번: 20143211

---

## 과제 목표: The Movies Data를 이용하여 비슷한 드라마 찾기

### The Movies Dataset
- 영화 평점 데이터셋으로, kaggle에서 제공하는 notebook 환경을 사용하거나 데이터셋을 다운로드할 수도 있다.

### 비슷한 영화 찾기
- 방법 및 순서
  
  1. Pandas를 이용하여 데이터 불러오기
  2. Pandas를 이용하여 데이터 정제하기
  3. Python의 집합을 활용하여 Jaccard 유사도 구하기
  4. Numpy를 이용하여 Pearson 유사도 구하기
  5. 모든 영화에 대해서 추천점수 계산하고, 가장 추천점수가 높은 영화 고르기

- Pandas
  - MS Office의 Excel과 같이 행과 열로 구성된 데이터 객체를 다루는 Python용 데이터분석 라이브러리이다.
  - Python 프로그래밍으로 데이터를 다룰 수 있다는 장점이 있다.

### Pandas 사용하기
- Pandas, Numpy 패키지 import

In [53]:
import numpy as np
import pandas as pd

### Pandas로 Metadata 읽고 정제하기
- 과제를 진행할 때에는 kaggle에서 데이터셋을 드라이브에 다운로드 받아 사용하였다.
- 데이터셋을 불러오기 위해 드라이브를 마운트한다.

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 아래의 코드는 다운로드 받은 데이터셋이 위치한 폴더 내에 있는 csv 파일들을 확인하는 코드이다.
- The movies dataset에 있는 다양한 데이터가 존재하는 것을 알 수 있다.

In [55]:
import os
for dirname, _, filenames in os.walk('/content/drive/My Drive/datasets/kaggle/The_movies_dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/My Drive/datasets/kaggle/The_movies_dataset/credits.csv
/content/drive/My Drive/datasets/kaggle/The_movies_dataset/keywords.csv
/content/drive/My Drive/datasets/kaggle/The_movies_dataset/links_small.csv
/content/drive/My Drive/datasets/kaggle/The_movies_dataset/links.csv
/content/drive/My Drive/datasets/kaggle/The_movies_dataset/movies_metadata.csv
/content/drive/My Drive/datasets/kaggle/The_movies_dataset/ratings.csv
/content/drive/My Drive/datasets/kaggle/The_movies_dataset/ratings_small.csv


1. metadata 읽기
  - csv 파일을 Pandas로 불러올 때는 read_csv 함수를 사용한다.
  - 위의 파일 경로를 사용하여 movies_metadata.csv 파일을 불러온다.
  - 아래와 같이 meta에 metadata가 입력된 것을 확인할 수 있다.

In [56]:
meta = pd.read_csv("/content/drive/My Drive/datasets/kaggle/The_movies_dataset/movies_metadata.csv")
meta

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,Fa

2. metadata에서 필요한 열(column)만 추려내기
  - 아래와 같이 열의 이름을 통해 필요한 열만 추려낼 수 있다.
  - 하나의 열만 추려낼 때에는 대괄호 안에 열 이름만 써주면 되지만, 여러 개의 열을 추릴 때에는 아래와 같이 리스트 내에 열 이름을 써주어야 한다.
  - 출력 결과에서 기존의 meta에서 필요한 열만 불러온 것을 알 수 있다.

In [57]:
meta = meta[['id', 'original_title', 'original_language', 'genres']]
meta

,id,original_title,original_language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...
45461,439050,رگ خواب,fa,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n..."
45462,111109,Siglo ng Pagluluwal,tl,"[{'id': 18, 'name': 'Drama'}]"
45463,67758,Betrayal,en,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,Satana likuyushchiy,en,[]


3. 열(column) 이름 변경하기
  - Pandas의 rename 메소드를 사용하여 데이터프레임(DataFrame) 열의 이름을 변경할 수 있다.
  - 열 이름을 변경할 때에는 columns, 행 이름을 변경할 때에는 index 파라미터를 사용하면 된다.
  - 출력 결과를 보면, 열의 이름이 변경된 것을 알 수 있다.

In [58]:
meta = meta.rename(columns={'id':'movieId', 'original_title':'title', 'original_language':'language'})
meta

,movieId,title,language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...
45461,439050,رگ خواب,fa,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n..."
45462,111109,Siglo ng Pagluluwal,tl,"[{'id': 18, 'name': 'Drama'}]"
45463,67758,Betrayal,en,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,Satana likuyushchiy,en,[]


4. language가 'en'인 데이터만 솎아내기
  - Pandas의 loc는 데이터프레임의 데이터에 접근할 때 사용하는 메서드이다.
  - loc에 입력할 수 있는 유형은 다음과 같다.
    1. 행/열의 이름
    2. :
    3. 리스트(ex. ['movieId', 'title']) - 여러 개의 행/열을 불러올 수 있다.
    4. Boolean 값을 가지는 Series 벡터
  - 아래의 코드는 meta.language가 'en'일 때 True 값을 가지는 Series 벡터를 loc에 입력한 형태이다.
  - 이를 통해 해당 행이 True 값을 가지는 데이터만 불러올 수 있다.
  - 출력 결과를 통해 language가 'en'인 데이터만 솎아진 것을 알 수 있다.

In [59]:
meta = meta.loc[meta.language=='en', :]
meta

,movieId,title,language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...
45459,222848,Caged Heat 3000,en,"[{'id': 878, 'name': 'Science Fiction'}]"
45460,30840,Robin Hood,en,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name..."
45463,67758,Betrayal,en,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,Satana likuyushchiy,en,[]


5. movieId의 타입을 숫자로 변경하기
  - 아래의 출력을 보면, movieId의 dtype이 object인 것을 알 수 있다.
  - 이를 숫자 데이터형으로 변경해준다.

In [60]:
meta.movieId

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45459    222848
45460     30840
45463     67758
45464    227506
45465    461257
Name: movieId, Length: 32269, dtype: object

 - 데이터를 숫자 데이터형으로 사용할 때는 pd.to_numeric 메서드를 사용한다.
 - 출력을 보면, dtype이 int64로 변경된 것을 알 수 있다.

In [61]:
meta.movieId = pd.to_numeric(meta.movieId)
meta.movieId

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45459    222848
45460     30840
45463     67758
45464    227506
45465    461257
Name: movieId, Length: 32269, dtype: int64

6-1. genre 정제하기: json string을 python set으로 변환하는 함수 구현
  - 아래의 출력을 보면, genre의 데이터가 json string의 형식으로 입력되어 있는 것을 알 수 있다.
  - 이를 python set으로 변환하는 함수를 구현하여 genres의 모든 데이터를 python set으 변환해준다.

In [62]:
meta.genres[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

 - 아래의 함수는 json string을 python set으로 변환하는 함수이다.
 - eval() 함수는 string으로 되어 있는 파이썬 코드를 실행시켜주는 함수이다.
 - eval() 함수를 통해 각 데이터를 for문으로 불러오고,  key 값 'name'을 입력하여 사용할 데이터인 장르명을 genre_set에 추가한다.

In [63]:
def str_to_set(x):
  genre_set = set()
  for item in eval(x):
    genre_set.add(item['name'])
  return genre_set

6-2. genre 정제하기: meta.genres의 모든 값에 str_to_set 적용
  - apply 메서드를 사용하여 위에서 구현한 함수 str_to_set을 meta.genres의 모든 값에 행 단위로 적용할 수 있다.
  - 출력 결과를 보면, genres 열에 장르명만 입력된 것을 확인할 수 있다.

In [64]:
meta.genres = meta.genres.apply(str_to_set)
meta

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,movieId,title,language,genres
0,862,Toy Story,en,"{Comedy, Animation, Family}"
1,8844,Jumanji,en,"{Adventure, Family, Fantasy}"
2,15602,Grumpier Old Men,en,"{Comedy, Romance}"
3,31357,Waiting to Exhale,en,"{Comedy, Drama, Romance}"
4,11862,Father of the Bride Part II,en,{Comedy}
...,...,...,...,...
45459,222848,Caged Heat 3000,en,{Science Fiction}
45460,30840,Robin Hood,en,"{Action, Drama, Romance}"
45463,67758,Betrayal,en,"{Thriller, Action, Drama}"
45464,227506,Satana likuyushchiy,en,{}


### Pandas로 Keywords 읽고 정제하기
1. keywords 데이터 읽기
  - read_csv 함수를 사용하여 keywords.csv 파일을 불러온다.
  - 아래와 같이 keywords에 keywords 데이터가 입력된 것을 확인할 수 있다.

In [65]:
keywords = pd.read_csv("/content/drive/My Drive/datasets/kaggle/The_movies_dataset/keywords.csv")
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


2. keywords 데이터의 keywords를 집합으로 변환하기
 - keywords의 keywords 데이터도 meta.genres와 같이 json string의 형식으로 입력되어 있으므로, 같은 방법으로 str_to_set 함수를 적용하여 데이터를 python set으로 변환한다.

In [66]:
keywords.keywords = keywords.keywords.apply(str_to_set)
keywords

,id,keywords
0,862,"{toy, rivalry, boy, friends, boy next door, ne..."
1,8844,"{board game, disappearance, recluse, giant ins..."
2,15602,"{old men, best friend, fishing, duringcreditss..."
3,31357,"{chick flick, single mother, interracial relat..."
4,11862,"{aging, contraception, baby, daughter, mother ..."
...,...,...
46414,439050,{tragic love}
46415,111109,"{pinoy, artist, play}"
46416,67758,{}
46417,227506,{}


3. id 열을 movieId로 이름 변경하고 데이터 타입을 숫자로 변환하기
 - rename 메서드를 사용하여 열의 이름을 변경하고, to_numeric 메서드를 사용하여 dtype이 숫자가 되도록 변환한다.

In [67]:
keywords = keywords.rename(columns={'id':'movieId'})
keywords.movieId = pd.to_numeric(keywords.movieId)
keywords

,movieId,keywords
0,862,"{toy, rivalry, boy, friends, boy next door, ne..."
1,8844,"{board game, disappearance, recluse, giant ins..."
2,15602,"{old men, best friend, fishing, duringcreditss..."
3,31357,"{chick flick, single mother, interracial relat..."
4,11862,"{aging, contraception, baby, daughter, mother ..."
...,...,...
46414,439050,{tragic love}
46415,111109,"{pinoy, artist, play}"
46416,67758,{}
46417,227506,{}


### Keywords와 Metadata 합치기
1. 같은 movieId를 갖는 데이터를 합치기
 - pd.merge 메서드를 사용하면 데이터프레임의 공통 요소를 기준으로 두 개의 데이터프레임을 합칠 수 있다.
 - on 파라미터에 공통되는 열의 이름을 입력한다('movieId').
 - how는 어떤 방식으로 데이터프레임을 합칠지 지정하는 파라미터이다.
 - how 파라미터에는 'inner', 'outer', 'left', 'right'를 입력할 수 있는데, 그중 'inner' 방식은 두 데이터프레임에 공통으로 존재하는 데이터(교집합 형태)만 남도록 합친다.
 - 출력 결과를 보면, movieId를 기준으로 meta에 keywords가 합쳐진 것을 알 수 있다.

In [68]:
meta = pd.merge(meta, keywords, on='movieId', how='inner')
meta

,movieId,title,language,genres,keywords
0,862,Toy Story,en,"{Comedy, Animation, Family}","{toy, rivalry, boy, friends, boy next door, ne..."
1,8844,Jumanji,en,"{Adventure, Family, Fantasy}","{board game, disappearance, recluse, giant ins..."
2,15602,Grumpier Old Men,en,"{Comedy, Romance}","{old men, best friend, fishing, duringcreditss..."
3,31357,Waiting to Exhale,en,"{Comedy, Drama, Romance}","{chick flick, single mother, interracial relat..."
4,11862,Father of the Bride Part II,en,{Comedy},"{aging, contraception, baby, daughter, mother ..."
...,...,...,...,...,...
32847,222848,Caged Heat 3000,en,{Science Fiction},{}
32848,30840,Robin Hood,en,"{Action, Drama, Romance}",{}
32849,67758,Betrayal,en,"{Thriller, Action, Drama}",{}
32850,227506,Satana likuyushchiy,en,{},{}


### Jaccard Similarity
1. 영화 찾아보고 합쳐서 출력해보기
 - meta에서 영화 제목이 "The Dark Knight"와 "The Dark Knight Rises"인 데이터를 불러온다.
 - iloc 메서드는 loc과 비슷한 메서드로, loc과 다른 점은 iloc는 인덱스 값을 통해 데이터를 불러온다는 것이다.
 - 여기서는 영화 제목이 같은 데이터가 meta에 여러 개 존재할 수 있으므로, 그 중에서 첫번째 행의 데이터를 불러오기 위해 iloc 메서드를 사용하였다.
 - pd.concat 메서드는 행이나 열의 사이즈가 같은 데이터들을 합쳐준다.
 - 아래의 출력 결과는 두 영화의 데이터를 불러와 하나의 데이터프레임으로 합쳐서 출력한 것이다.

In [69]:
dk = meta.loc[meta.title == "The Dark Knight"].iloc[0]
dkr = meta.loc[meta.title == "The Dark Knight Rises"].iloc[0]
pd.concat([dk, dkr], axis=1).T

,movieId,title,language,genres,keywords
10278,155,The Dark Knight,en,"{Action, Crime, Drama, Thriller}","{criminal mastermind, gotham city, imax, sadis..."
14315,49026,The Dark Knight Rises,en,"{Crime, Action, Thriller, Drama}","{gotham city, terrorist, crime fighter, cover-..."


2. Jaccard 유사도 함수 구현 및 실행
 - Jaccard 유사도는 두 영화가 genres와 keywords 데이터를 얼마나 공유하는지를 통해 유사도를 계산한다.
 - 각각의 영화의 genres와 keywords를 합쳐 set으로 만든 후, 두 영화의 set의 교집합 크기를 set의 합집합 크기로 나누면 정량화된 수치로 Jaccard 유사도를 계산할 수 있다.
 - 아래의 코드는 Jaccard 유사도를 계산하는 함수를 구현한 것이다.

In [70]:
def jaccard_similarity(s1, s2):
  if len(s1 | s2) == 0: # 분모가 0이 되면 유사도를 계산하지 않고 0이 리턴되도록 한다.
    return 0
  return len(s1 & s2) / len(s1 | s2)

- 아래의 코드는 두 영화 The Dark Knight와 The Dark Knight Rises의 Jaccard 유사도를 계산한다.
- OR 연산을 통해 genres와 keywords의 합집합을 계산할 수 있다.
- 결과를 보면, The Dark Knight와 The Dark Knight Rises의 Jaccard 유사도는 약 0.3714인 것을 알 수 있다.

In [92]:
dk_set = dk.genres | dk.keywords
dkr_set = dkr.genres | dkr.keywords

jaccard_similarity(dk_set, dkr_set)

0.37142857142857144

- 아래의 코드는 The Dark Knight와 Sudden Death의 Jaccard 유사도를 계산한 것이다.
- 두 영화의 Jaccard 유사도는 약 0.0714이고, 이를 통해 The Dark Knight는 Sudden Death보다 The Dark Knight Rises와 더 비슷한 영화라는 것을 알 수 있다.

In [93]:
sd = meta.loc[meta.title=="Sudden Death"].iloc[0]
sd_set = sd.genres | sd.keywords

jaccard_similarity(dk_set, sd_set)

0.07142857142857142

### Rating 데이터 읽고 정제하기
1. Rating 데이터 읽기
  - read_csv 함수를 사용하여 ratings_small.csv 파일을 불러온다.
  - ratings.csv는 데이터의 크기가 너무 크므로 이번 과제에서는 ratings_small.csv에서 데이터를 불러와 사용한다.
  - 아래와 같이 ratings에 데이터가 입력된 것을 확인할 수 있다.

In [83]:
ratings = pd.read_csv("/content/drive/My Drive/datasets/kaggle/The_movies_dataset/ratings_small.csv")
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


2. Rating의 movieId를 숫자로 변환
 - pd.to_numeric 메서드를 사용하여 ratings.movieId의 dtype이 숫자가 되도록 변환한다.


In [84]:
ratings.movieId = pd.to_numeric(ratings.movieId)
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


3. metadata로부터 title 정보 가져와 합치기
 - pd.merge 메서드를 사용해 movieId 열을 기준으로 교집합하여 ratings에 metadata.title을 합친다.
 - meta에서 movieId와 title 열만 필요하므로 리스트 형식으로 meta를 불러온다.
 - 출력 결과를 보면 movieId에 따라 ratings에 title이 추가된 것을 알 수 있다.

In [86]:
ratings = pd.merge(ratings, meta[['movieId', 'title']], on="movieId", how='inner')
ratings

,userId,movieId,rating,timestamp,title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III
...,...,...,...,...,...
33295,652,127728,5.0,1439586990,8:46
33296,652,129009,4.0,1442690827,Love Is a Ball
33297,653,2103,3.0,948161066,Solaris
33298,659,167,4.0,836137550,K-PAX


4. pivot table 만들기
 - Pandas의 pivot_table 메서드는 데이터프레임으로부터 새로운 pivot table을 생성해준다.
 - index와 columns 파라미터에는 각각 행과 열로 사용할 데이터의 이름을, values 파라미터에는 pivot table의 값으로 사용할 데이터의 이름을 입력한다.
 - 출력 결과는 ratings로부터 userId, title, rating 데이터를 각각 행, 열, 테이블값으로 하여 pivot table을 생성한 것이다.

In [87]:
matrix = ratings.pivot_table(index='userId', columns='title', values='rating')
matrix

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,15 Minutes,16 Blocks,1984,2 Days in Paris,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,2061 - Un anno eccezionale,21 Grams,24 Hour Party People,25th Hour,28 Days Later,28 Weeks Later,29th Street,2:37,3 Ninjas: High Noon at Mega Mountain,300,40 Days and 40 Nights,42nd Street,48 Hrs.,4: Rise of the Silver Surfer,5 Card Stud,5 Days of War,50 First Dates,54,8 Mile,88 Minutes,8:46,9 Songs,95 Miles to Go,...,Wild Things,Wild Tigers I Have Known,Wild West Comedy Show: 30 Days & 30 Nights - Hollywood to the Heartland,Wild Wild West,Wild at Heart,Wild in the Country,Will Penny,Willy Wonka & the Chocolate Factory,Wish You Were Here,Within the Woods,Without You I'm Nothing,Wizards of Waverly Place: The Movie,Wizards of the Lost Kingdom,Wolves of Wall Street,Working Girl,World Trade Center,Wuthering Heights,X-Men Origins: Wolverine,X: The Unknown,Yamakasi - Les samouraïs des temps modernes,Yankee Doodle Dandy,You Are Not I,You Instead,You Only Live Twice,You're a Big Boy Now,"You, Me and Dupree",Young Adam,Young Black Stallion,Young Frankenstein,Young Mr. Lincoln,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

5. Pearson 유사도 함수 구현
 - Pearson 유사도는 두 벡터의 내적한 값을 두 벡터의 크기로 나누어 두 벡터 간의 유사도를 구하는 방법이다.
 - matrix에 있는 영화의 평점들은 모두 양수 값을 가지므로, 평점을 벡터화하여 Pearson 유사도를 구하면 실제로 유사하지 않은 두 영화에 대해서도 Pearson 유사도는 양수 값을 가지게 되어 '유사하지 않은'이 아니라 '약간 유사한'의 의미를 갖게 된다.
 - 따라서 Pearson 유사도를 구하기 전에 먼저 평점 벡터의 평균값을 벡터에서 빼주면 평균보다 낮은 평점은 음수 값을 가지게 되고, 이에 따라 유사하지 않은 두 영화에 대해 Pearson 유사도 또한 음수 값으로 계산된다.
 - 아래의 코드는 Pearson 유사도를 구하는 함수를 구현한 코드이다.

In [89]:
def pearson_similarity(u1, u2):
  u1_c = u1 - u1.mean() # 평점 벡터에서 평균값을 빼주어 평균보다 낮은 평점이 음수 값을 가지게 만든다.
  u2_c = u2 - u2.mean()

  denom = ((u1_c ** 2).sum() * (u2_c ** 2).sum()) ** 0.5 # 두 벡터의 크기를 곱한 값이 0이 아닌 경우에만 
                                                         # Pearson 유사도가 계산되도록 한다.
  if denom != 0:
    return (u1_c * u2_c).sum() / denom
  else:
    return 0

6. The Dark Knight와 Prom Night의 Pearson 유사도 구해보기
 - 두 영화 The Dark Knight와 Prom Night의 평점 데이터를 matrix에서 불러와 벡터로 만들고, 이를 함수에 입력하여 두 영화의 Pearson 유사도를 계산한다.
 - 출력 결과, 두 영화의 Pearson 유사도는 약 0.7736인 것을 알 수 있다.

In [91]:
dk_rating = matrix["The Dark Knight"]
pn_rating = matrix["Prom Night"]
pearson_similarity(dk_rating, pn_rating)

0.773565934694095

### 비슷한 영화 추천 기능 구현
- 아래의 코드는 위에서 구현한 Jaccard 유사도 계산 함수와 Pearson 유사도 계산 함수를 사용하여 입력한 영화(input_title)와 유사한 n개의 영화를 가장 유사한 영화부터 순서대로 리턴해주는 함수이다.
- 가중치 alpha에 따라 Jaccard 유사도와 Pearson 유사도의 가중치 합을 하여 최종 점수인 score를 계산하고, score에 따라 유사한 영화들을 순서대로 정렬한다.

In [110]:
def find_similar_movies(input_title, n, alpha):
  input_meta = meta.loc[meta.title==input_title].iloc[0] # 입력된 영화의 metadata
  input_set = input_meta.genres | input_meta.keywords    # 입력된 영화의 genres와 keywords의 합집합. Jaccard 유사도를 구할 때 사용됨.
 
  result = [] # 모든 영화마다 유사도를 계산하여 저장하기 위한 리스트
  
  for this_title in matrix.columns:
    if this_title == input_title: # 입력된 영화가 나오면 유사도를 계산하지 않고 넘어감.
      continue

    this_meta = meta.loc[meta.title==this_title].iloc[0] # 유사도를 계산하려는 영화의 metadata
    this_set = this_meta.genres | this_meta.keywords     # 이 영화의 genres와 keywords의 합집합. Jaccard 유사도를 구할 때 사용됨.

    pearson = pearson_similarity(matrix[input_title], matrix[this_title]) # 입력 영화와 이번 영화의 pearson 유사도 결과
    jaccard = jaccard_similarity(this_set, input_set)                     # 입력 영화와 이번 영화의 jaccard 유사도 결과

    score = alpha * pearson + (1 - alpha) * jaccard      # pearson 점수와 jaccard 점수의 가중치 합
    result.append((this_title, pearson, jaccard, score)) # result에 계산 결과 추가

  result.sort(key=lambda r:r[3], reverse=True) # 모든 영화에 대해 유사도 계산이 끝나면 result를 정렬
    
  return result[:n]                            # 상위 n개의 결과를 return

### 비슷한 영화 추천
1. The Dark Knight와 비슷한 영화 추천해보기
  - 위에서 구현한 함수를 사용하여 The Dark Knight와 비슷한 영화를 순서대로 구한다.
  - pd.DataFrame 메서드를 사용하면 출력 결과를 데이터프레임 형태로 깔끔하게 확인할 수 있다.
  - index나 columns 파라미터를 사용하면 행/열의 이름을 지정할 수 있다.
  - 출력 결과는 The Dark Knight와 가장 유사한 n=10개의 영화가 나타난 것이다.
  - alpha=0.5로 Pearson 유사도와 Jaccard 유사도가 동일한 비중으로 score가 계산된 결과이고, 영화 Wild Wild West가 score = 0.4029로 The Dark Knight와 가장 비슷한 영화라는 것을 알 수 있다.

In [113]:
result = find_similar_movies("The Dark Knight", 10, 0.5)
pd.DataFrame(result, columns=['title', 'Pearson', 'Jaccard', 'score'])

,title,Pearson,Jaccard,score
0,Wild Wild West,0.773566,0.032258,0.402912
1,Prom Night,0.773566,0.022222,0.397894
2,Ludwig,0.472718,0.033333,0.253026
3,Yamakasi - Les samouraïs des temps modernes,0.377145,0.125000,0.251072
4,El asaltante,0.446949,0.040000,0.243475
5,Midnight in the Garden of Good and Evil,0.373841,0.111111,0.242476
6,Sneakers,0.415830,0.068966,0.242398
7,Topaz,0.377145,0.103448,0.240296
8,Godzilla,0.413141,0.052632,0.232886
9,Bushwhacked,0.396081,0.064516,0.230299


 - 아래의 두 코드는 각각 alpha=0.3으로 하여 Jaccard 유사도의 비중을 더 크게 두고 score를 계산한 결과와 alpha=0.7으로 하여 Pearson 유사도의 비중을 더 크게 두고 score를 계산한 결과를 출력한 것이다.
 - 결과를 보면, alpha=0.5일 때와 비교했을 때 The Dark Knight와 유사한 1순위와 2순위 영화를 제외하면  영화의 순위가 약간 바뀌거나 새로운 영화가 추가되는 것을 알 수 있다.

In [117]:
result = find_similar_movies("The Dark Knight", 10, 0.3)
pd.DataFrame(result, columns=['title', 'Pearson', 'Jaccard', 'score'])

,title,Pearson,Jaccard,score
0,Wild Wild West,0.773566,0.032258,0.254650
1,Prom Night,0.773566,0.022222,0.247625
2,Batman Begins,0.005015,0.292683,0.206383
3,Yamakasi - Les samouraïs des temps modernes,0.377145,0.125000,0.200643
4,Blue Thunder,0.326617,0.133333,0.191318
5,Midnight in the Garden of Good and Evil,0.373841,0.111111,0.189930
6,Topaz,0.377145,0.103448,0.185557
7,Big Bad Mama,0.344649,0.107143,0.178395
8,Sneakers,0.415830,0.068966,0.173025
9,The Enforcer,0.326617,0.103448,0.170399


In [115]:
result = find_similar_movies("The Dark Knight", 10, 0.7)
pd.DataFrame(result, columns=['title', 'Pearson', 'Jaccard', 'score'])

,title,Pearson,Jaccard,score
0,Wild Wild West,0.773566,0.032258,0.551174
1,Prom Night,0.773566,0.022222,0.548163
2,Ludwig,0.472718,0.033333,0.340903
3,El asaltante,0.446949,0.040000,0.324864
4,Sneakers,0.415830,0.068966,0.311771
5,Godzilla,0.413141,0.052632,0.304988
6,Yamakasi - Les samouraïs des temps modernes,0.377145,0.125000,0.301501
7,Bushwhacked,0.396081,0.064516,0.296612
8,Topaz,0.377145,0.103448,0.295036
9,Midnight in the Garden of Good and Evil,0.373841,0.111111,0.295022


- 아래의 코드는 영화 Rocky III와 비슷한 영화를 출력한 것이다.
- 결과를 보면, 비슷한 영화 순위 상위권에 Rocky 시리즈가 4편(Rocky V, Rocky II, Rocky Balboa, Rocky IV)이나 있는 것을 확인할 수 있고, 이를 통해 find_similar_movies 함수가 Jaccard 유사도와 Pearson 유사도를 이용하여 비슷한 영화를 잘 찾아낸다는 것을 알 수 있다.

In [125]:
result = find_similar_movies("Rocky III", 10, 0.5)
pd.DataFrame(result, columns=['title', 'Pearson', 'Jaccard', 'score'])

,title,Pearson,Jaccard,score
0,Rocky V,0.297157,0.272727,0.284942
1,Rocky II,0.182715,0.333333,0.258024
2,Rocky Balboa,0.152681,0.357143,0.254912
3,Rocky IV,0.117354,0.346154,0.231754
4,About Schmidt,0.295264,0.040000,0.167632
5,54,0.293060,0.041667,0.167363
6,Cinderella Man,0.030373,0.296296,0.163335
7,Michael,0.287140,0.038462,0.162801
8,I'll Sleep When I'm Dead,0.223659,0.090909,0.157284
9,Touch of Evil,0.280075,0.034483,0.157279
